# Experimento 1: Palavras e temas dominantes


In [ ]:
# 1. Instalação de pacotes que podem não vir por padrão (normalmente o Colab já tem, mas garante)
!pip install wordcloud nltk pandas matplotlib scikit-learn

# 2. Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive

# 3. Configurações de Visualização
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

# 4. Montar o Google Drive
drive.mount('/content/drive')

# 5. Carregar os Dados
path_folder = '/content/drive/MyDrive/IFES/Disciplinas/6-periodo/Topicos-Especiais-de-Banco-de-Dados/Jogo-complete-a-letra/albuns_engenheiros_csv/musicas-analise'
all_files = glob.glob(os.path.join(path_folder, "*.csv"))

df_list = []

print(f"Encontrados {len(all_files)} arquivos de álbuns.")

for filename in all_files:
    try:
        # Tenta ler com utf-8, se falhar tenta latin-1 (comum em arquivos excel/csv antigos)
        try:
            df_temp = pd.read_csv(filename, encoding='utf-8')
        except UnicodeDecodeError:
            df_temp = pd.read_csv(filename, encoding='latin-1')

        # Padronizar nomes das colunas para minúsculas para evitar erros (Titulo vs titulo)
        df_temp.columns = df_temp.columns.str.lower()

        # Verifica se as colunas necessárias existem
        if 'titulo' in df_temp.columns and 'letra' in df_temp.columns:
            df_list.append(df_temp[['titulo', 'letra']])
        else:
            print(f"Aviso: Arquivo {os.path.basename(filename)} ignorado (colunas incorretas).")

    except Exception as e:
        print(f"Erro ao ler {filename}: {e}")

# Concatenar tudo em um único DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Total de músicas carregadas: {len(df)}")
df.head()

In [ ]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string

# Configurações iniciais
nltk.download('stopwords')
nltk.download('punkt')

# --- 1. Carregamento e Pré-processamento [cite: 15] ---
# SUBSTITUA 'seu_arquivo.csv' PELO CAMINHO DO SEU DATASET REAL
# O dataset deve ter uma coluna com o texto das letras, aqui chamada de 'letra'
try:
    df = pd.read_csv('seu_arquivo.csv')
except:
    # Dados fictícios para exemplo caso não tenha o arquivo agora
    data = {'titulo': ['Infinita Highway', 'Refrão de Bolero'],
            'letra': ['Você me faz correr demais nesta estrada', 'Um dia me disseram que as nuvens não eram de algodão']}
    df = pd.DataFrame(data)

def preprocessar(texto):
    # Converter para minúsculas [cite: 17]
    texto = texto.lower()
    # Remover pontuação [cite: 17]
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Tokenização simples (split)
    tokens = texto.split()
    # Remover Stopwords [cite: 18]
    stops = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stops]
    return tokens

# Aplicar pré-processamento
df['tokens'] = df['letra'].apply(preprocessar)
df['texto_limpo'] = df['tokens'].apply(lambda x: ' '.join(x))

# --- 2. Análise 1: Contagem Simples e WordCloud [cite: 19] ---
todos_tokens = [word for lista in df['tokens'] for word in lista]
contagem = Counter(todos_tokens)

# Top-20 Termos [cite: 22]
print("--- TOP 20 PALAVRAS MAIS COMUNS ---")
print(contagem.most_common(20))

# Nuvem de Palavras [cite: 21]
wc = WordCloud(width=800, height=400, background_color='white', max_words=100).generate_from_frequencies(contagem)
plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("Nuvem de Palavras - Engenheiros do Hawaii")
plt.show()

# --- 3. Análise 2: TF-IDF [cite: 23] ---
# Identificar palavras importantes para músicas específicas [cite: 25]
tfidf = TfidfVectorizer(stop_words=list(stopwords.words('portuguese')))
matrix_tfidf = tfidf.fit_transform(df['texto_limpo'])
feature_names = tfidf.get_feature_names_out()

# Média dos scores TF-IDF para achar termos relevantes globalmente [cite: 27]
mean_tfidf = matrix_tfidf.mean(axis=0)
tfidf_scores = [(feature_names[i], mean_tfidf[0, i]) for i in range(mean_tfidf.shape[1])]
tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

print("\n--- TOP 20 TERMOS (SCORE TF-IDF) ---")
for termo, score in tfidf_scores[:20]:
    print(f"{termo}: {score:.4f}")

#Experimento 2: Geração no Estilo Engenheiros

Resultado Gerado (Modelo 1 - Gemini):

Título: O Algoritmo da Solidão

No espelho do elevador, o reflexo não tem wi-fi Subindo pro andar errado, onde o ego se distrai Li em algum muro branco, ou foi num feed qualquer? Que a gente busca sentido onde só existe "o que vier"

(Refrão) Somos dados viciados rolando num tabuleiro digital Procurando a saída de emergência no meio do carnaval É tudo tão moderno, é tudo tão medieval A gente curte a foto e ignora o vendaval

O filósofo de bar disse que o tempo é relativo Mas o boleto vence dia 10, imperativo Entre Camus e o Big Brother, escolhemos o controle remoto A vida passa em 4K, mas a alma saiu da foto.

In [ ]:
# Exemplo genérico para chamar um modelo local (Llama 3 ou Mistral) [cite: 37]
# Requer instalação de biblioteca específica ou setup do Ollama
import requests
import json

prompt = """
Aja como Humberto Gessinger. Escreva uma letra de música no estilo da banda Engenheiros do Hawaii.
Use ironia, palavras como 'estrada', 'muro', 'espelho' e temas existenciais.
"""

# Exemplo de chamada para API local do Ollama (se estiver rodando na porta 11434)
url = "http://localhost:11434/api/generate"
data = {
    "model": "llama3",
    "prompt": prompt,
    "stream": False
}

try:
    response = requests.post(url, json=data)
    resultado = response.json()['response']
    print("--- LETRA GERADA PELO MODELO 2 (OPEN SOURCE) ---")
    print(resultado)
except:
    print("Erro ao conectar com o modelo local. Certifique-se que o Ollama está rodando.")

#Experimento 3: Classificação Intuitiva

In [ ]:
import pandas as pd

# Carregar dados novamente (ou usar o df do experimento 1)
# df = pd.read_csv('seu_arquivo.csv')

# 1. Definição dos Léxicos [cite: 58, 59, 60]
list_melancolicas = ['triste', 'dor', 'sozinho', 'saudade', 'noite', 'fim', 'escuro', 'lágrima', 'adeus', 'vazio']
list_otimistas = ['sol', 'luz', 'novo', 'dia', 'amanhã', 'esperança', 'sorriso', 'vida', 'amor', 'futuro']
list_filosoficas = ['infinito', 'razão', 'ser', 'mundo', 'porquê', 'tempo', 'verdade', 'sentido', 'dúvida', 'estrada']

# 2. Desenvolvimento do Classificador [cite: 62]
def classificar_letra(tokens):
    # Contadores
    score_melancolia = 0
    score_otimismo = 0
    score_filosofia = 0

    # Contagem [cite: 64]
    for palavra in tokens:
        if palavra in list_melancolicas:
            score_melancolia += 1
        elif palavra in list_otimistas:
            score_otimismo += 1
        elif palavra in list_filosoficas:
            score_filosofia += 1

    # Decisão [cite: 65]
    scores = {
        'Melancólica': score_melancolia,
        'Otimista': score_otimismo,
        'Filosófica': score_filosofia
    }

    # Pega a categoria com maior valor. Se empate, pega a primeira.
    categoria_vencedora = max(scores, key=scores.get)

    # Tratamento para quando nenhuma palavra é encontrada (opcional)
    if scores[categoria_vencedora] == 0:
        return 'Indefinido'

    return categoria_vencedora

# Aplicar classificador [cite: 70]
# Certifique-se que a coluna 'tokens' existe (do Exp 1)
df['classificacao_intuitiva'] = df['tokens'].apply(classificar_letra)

# 3. Resultados Quantitativos [cite: 71]
distribuicao = df['classificacao_intuitiva'].value_counts(normalize=True) * 100
print("--- DISTRIBUIÇÃO DAS CATEGORIAS (%) ---")
print(distribuicao)

# Gráfico de Pizza
distribuicao.plot(kind='pie', autopct='%1.1f%%', figsize=(8, 8))
plt.title("Classificação Intuitiva das Letras")
plt.ylabel('')
plt.show()

# 4. Avaliação Qualitativa [cite: 72]
# Verificar músicas específicas
musicas_teste = ["Infinita Highway", "Pra Ser Sincero", "O Papa é Pop"]
print("\n--- VALIDAÇÃO MANUAL ---")
for musica in musicas_teste:
    resultado = df[df['titulo'] == musica]['classificacao_intuitiva']
    if not resultado.empty:
        print(f"Música: {musica} -> Classificação: {resultado.values[0]}")